This is my optimized code for calculating the cosine similarity between two text documents. Fundamentally, I used Counter to significantly reduce the processing time, and also use .extend() instead of concatenating two lists.

In [1]:
import math
import string
import sys
import re
from collections import Counter


def read_file(filename):
    """Read a .txt file and return a sorted dictionary of all the unique words in that file,
    sorted alphabetically. The value of each word is the number of occurences for that word in the file.
    """
    try:
        fp = open(filename)
        L = fp.readlines()
        word_list = []
        sorted_dict = {}
        for line in L:
            words_in_line = [x.lower() for x in re.split("[^A-Za-z0-9]",line) if x]
            word_list.extend(words_in_line)
        unsorted_dict = dict(Counter(word_list))
        sorted_keys = sorted(unsorted_dict.keys())
        for key in sorted_keys:
            sorted_dict[key] = unsorted_dict[key]
    except IOError as excObj:
        print(str(excObj))
        print("Error opening or reading input file: " + filename)
        sys.exit()
    return sorted_dict


def inner_product(D1,D2):
    """Calculate the inner product between two vectors, 
    in this case two documents represented as two sorted dictionaries.
    """
    sum = 0.0
    i = 0
    j = 0
    L1 = list(D1.keys())
    L2 = list(D2.keys())
    while i<len(L1) and j<len(L2):
        if L1[i] == L2[j]:
            # both vectors have this word
            sum += D1[L1[i]] * D2[L2[j]]
            i += 1
            j += 1
        elif L1[i] < L2[j]:
            # word L1[i] is in L1 but not L2
            i += 1
        else:
            # word L2[j] is in L2 but not L1
            j += 1
    return sum

def vector_angle(D1,D2):
    numerator = inner_product(D1,D2)
    denominator = math.sqrt(inner_product(D1,D1)*inner_product(D2,D2))
    return numerator/denominator

def document_similarity(filename_1, filename_2, verbose=True):
    sorted_word_1 = read_file(filename_1)
    sorted_word_2 = read_file(filename_2)
    cosine = vector_angle(sorted_word_1,sorted_word_2)
    # Use f-strings; see https://realpython.com/python-f-strings/ for more information
    if verbose:
        print(f"The cosine between the documents is {cosine : 0.6f}.")
        print(f"The angle between the documents is {math.acos(cosine) : 0.6f} radians or {math.acos(cosine)*180/math.pi : .0f} degrees.")

In [2]:
document_similarity('data/t5.churchill.txt','data/t8.shakespeare.txt')

The cosine between the documents is  0.895120.
The angle between the documents is  0.462095 radians or  26 degrees.


In [3]:
%timeit document_similarity('data/t5.churchill.txt','data/t8.shakespeare.txt',verbose=False)

3.06 s ± 250 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The time taken is somewhere in the 2.5-3s region for a 10MB document, but I'm not overly happy with this, knowing I can bring it down further. Here are some ideas from MADS classmates, one I picked particularly for the same use of Counter.

In [4]:
from functools import lru_cache

def read_file_2(filename1,filename2):
    try:
        with open(filename1) as fp1, open(filename2) as fp2:
            L1 = fp1.readlines()
            L2 = fp2.readlines()
    except IOError as excObj:
        print(str(excObj))
        print("Error opening or reading input file: " + filename)
        sys.exit()
    return L1, L2

@lru_cache(maxsize=None)
def get_words_from_string(line):
    word_list = []          # accumulates words in line
    character_list = []     # accumulates characters in word
    for c in line:
        if c.isalnum():
            character_list.append(c)
        elif len(character_list)>0:
            word = str.join("", character_list)
            word = str.lower(word)
            word_list.append(word)
            character_list = []
    if len(character_list)>0:
        word = str.join("", character_list)
        word = str.lower(word)
        word_list.append(word)
    return word_list

def get_words_from_line_list(L):
    word_list = []
    for line in L:
        words_in_line = get_words_from_string(line)
        if not word_list:
            for words in words_in_line:
                word_list.append(words)
        else:
            word_list.extend(words_in_line)
    return Counter(word_list) #interesting use of counter here!

def vector_angle(c1,c2):
    # new syntax that I have never used. also, interesting way to calc vector angle using Counter
    numerator = sum(c1[n]*c2[n] for n in c1)
    denominator = math.sqrt(sum(c1[n]*c1[n] for n in c1)*sum(c2[n]*c2[n] for n in c2))
    return numerator/denominator

def document_similarity_2(filename1,filename2,verbose=True):
    file1, file2 = read_file_2(filename1,filename2)
    counter1 = get_words_from_line_list(file1)
    counter2 = get_words_from_line_list(file2)
    cosine = vector_angle(counter1,counter2)
    if verbose:
        print("File",filename1,":", len(file1),"lines,", sum(counter1.values()),"words,", len(counter1),"distinct words")
        print("File",filename2,":", len(file2),"lines,", sum(counter2.values()),"words,", len(counter2),"distinct words")
        print(f"The cosine between the documents is {cosine : 0.6f}.")
        print(f"The angle between the documents is {math.acos(cosine) : 0.6f} radians or {math.acos(cosine)*180/math.pi : .0f} degrees.")

In [5]:
document_similarity_2('data/t5.churchill.txt','data/t8.shakespeare.txt')

File data/t5.churchill.txt : 189685 lines, 1717247 words, 32544 distinct words
File data/t8.shakespeare.txt : 124456 lines, 929462 words, 23881 distinct words
The cosine between the documents is  0.895120.
The angle between the documents is  0.462095 radians or  26 degrees.


In [6]:
%timeit document_similarity_2('data/t5.churchill.txt','data/t8.shakespeare.txt',verbose=False)

726 ms ± 25.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


It's down to miliseconds ! 